In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import cv2

In [ ]:
train_dataset = r'/content/drive/MyDrive/COVID 19/Train'
categories = ['COVID','Lung_Opacity','Normal','Viral Pneumonia']
data_1 =[]
IMZ_SIZE = 180
for category in categories:
  folder_1 = os.path.join(train_dataset,category)
  label_1 = categories.index(category)
  for imz in os.listdir(folder_1):
    imz_join = os.path.join(folder_1,imz)
    imz_read = cv2.imread(imz_join)
    imz_resize = cv2.resize(imz_read,(IMZ_SIZE,IMZ_SIZE))
    data_1.append([imz_resize,label_1])


In [ ]:
test_dataset = r'/content/drive/MyDrive/COVID 19/Test'
data_2 = []
for category in categories:
  folder_2 = os.path.join(test_dataset,category)
  label_2 = categories.index(category)
  for imz_1 in os.listdir(folder_2):
    imz_join2 = os.path.join(folder_2,imz_1)
    imz_read2 = cv2.imread(imz_join2)
    imz_resize1 = cv2.resize(imz_read2,(IMZ_SIZE,IMZ_SIZE))
    data_2.append([imz_resize1,label_2])


In [ ]:
#for training data
import random
random.shuffle(data_1)

In [ ]:
import numpy as np
X_train = []
Y_train = []
for features,labels in data_1:
  X_train.append(features)
  Y_train.append(labels)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
Y_train_reshape = Y_train.reshape(len(Y_train),1*1)
Y_train_reshape[:5]

array([[2],
       [2],
       [0],
       [3],
       [2]])

In [ ]:
#categorical values
Y_train_categorical = keras.utils.to_categorical(Y_train_reshape,num_classes=4,dtype='float32')
Y_train_categorical[:5]

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]], dtype=float32)

In [ ]:
random.shuffle(data_2)

In [ ]:
X_test = []
Y_test = []
for features,labels in data_2:
  X_test.append(features)
  Y_test.append(labels)
X_test = np.array(X_test)
Y_test = np.array(Y_test)


In [ ]:
Y_test_reshape = Y_test.reshape(len(Y_test),1*1)
Y_test_categorical = keras.utils.to_categorical(Y_test_reshape,num_classes=4,dtype='float32')


In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
conv_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(180,180,3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [ ]:
model = keras.Sequential([
      conv_base,
      keras.layers.Flatten(),
      keras.layers.Dense(500,activation='relu'),
      keras.layers.Dropout(0.3),
      keras.layers.Dense(300,activation='relu'),
      keras.layers.Dropout(0.3),
      keras.layers.Dense(200,activation='relu'),
      keras.layers.Dense(4,activation='softmax'),
])




In [ ]:
conv_base.trainable=False

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train_categorical,epochs=20,validation_data=(X_test,Y_test_categorical),batch_size=5)

Epoch 1/20
3360/3360 [==============================] - 138s 37ms/step - loss: 1.3106 - accuracy: 0.7700 - val_loss: 0.3728 - val_accuracy: 0.8745
Epoch 2/20
3360/3360 [==============================] - 127s 38ms/step - loss: 0.4658 - accuracy: 0.8486 - val_loss: 0.3629 - val_accuracy: 0.8802
Epoch 3/20
3360/3360 [==============================] - 128s 38ms/step - loss: 0.4075 - accuracy: 0.8733 - val_loss: 0.4163 - val_accuracy: 0.8821
Epoch 4/20
3360/3360 [==============================] - 129s 38ms/step - loss: 0.3544 - accuracy: 0.8878 - val_loss: 0.3730 - val_accuracy: 0.8908
Epoch 5/20
3360/3360 [==============================] - 108s 32ms/step - loss: 0.3477 - accuracy: 0.8964 - val_loss: 0.3283 - val_accuracy: 0.8908
Epoch 6/20
3360/3360 [==============================] - 110s 33ms/step - loss: 0.3387 - accuracy: 0.9011 - val_loss: 0.3074 - val_accuracy: 0.9018
Epoch 7/20
3360/3360 [==============================] - 110s 33ms/step - loss: 0.3376 - accuracy: 0.8980 - val_loss: 0

In [ ]:
input_image = input('Enter the path of image:')
input_path = cv2.imread(input_image)
input_resize = cv2.resize(input_path,(180,180))
input_reshape = input_resize.reshape(1,180,180,3)
print(categories[np.argmax(model.predict(input_reshape))])



Enter the path of image:/content/drive/MyDrive/x-ray-COVID-19-header-1920x1080_1800000007267872.jpg
1/1 [==============================] - 1s 748ms/step
COVID


In [ ]:
#saving the model
model.save('/content/drive/MyDrive/saved model/model.h5')

In [ ]:
loaded_model = keras.models.load_model('model.h5')
loaded_model.load_weights('model.h5')
categories = ['COVID','Lung_Opacity','Normal','Viral Pneumonia']


In [ ]:
input_image = input('Enter the path of image:')
input_path = cv2.imread(input_image)
input_resize = cv2.resize(input_path,(180,180))
input_reshape = input_resize.reshape(1,180,180,3)
print(categories[np.argmax(loaded_model.predict(input_reshape))])



Enter the path of image:/content/drive/MyDrive/x-ray-COVID-19-header-1920x1080_1800000007267872.jpg
1/1 [==============================] - 0s 21ms/step
COVID
